In [34]:
from pathlib import Path
import pandas as pd
import os, pdb
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Mount google drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
#output file path
DATA_PATH = Path('/content/drive/MyDrive/vcf/')
#input file path
DATA_PATH_in = Path('/content/drive/MyDrive/vcf//input')

Mounted at /content/drive


In [7]:
files = os.listdir(DATA_PATH_in)

In [11]:
#creating .csv files and save in output directory
def save_dataframe(files):
  for f in files:
    print(f"Working with file {f}")
    with open(DATA_PATH_in / f) as _f:
      lines = _f.readlines()[8:]
    for i, line in enumerate(lines):
      line = [x.replace("\n", "") for x in line.split(" ")]
      if i == 0:
        df = pd.DataFrame(columns=line[2:])
      else:
        df.loc[i-1] = line
    _f = f+".csv"
    df.to_csv(DATA_PATH / "output" / _f, index=False)

In [ ]:
save_dataframe(files)

In [48]:
#Defining VACF function
def get_v_cf(f1, f2):
  f1 = pd.read_csv(DATA_PATH / "output" / f1)
  f2 = pd.read_csv(DATA_PATH / "output" / f2)

  v_numerator = ( f1.vx * f2.vx ) + ( f1.vy * f2.vy ) + ( f1.vz * f2.vz )
  v_denomenator = ( f1.vx * f1.vx ) + ( f1.vy * f1.vy ) + ( f1.vz * f1.vz )
  v_cf = np.sum(v_numerator)/np.sum(v_denomenator)

  return v_cf

In [49]:
#calculating VACF
v_cf = []
#time steps and dt
for i in range(0, 1000, 10):
  fname = f"dump.Ta_3.304_300K.{i}.csv"
  if i == 0:
    f_old = fname
    f_new = fname
    _v_cf = get_v_cf(f_old, f_new)
    v_cf.append(_v_cf)
  else:
    f_new = fname
    _v_cf = get_v_cf(f_old, f_new)
    v_cf.append(_v_cf)
    f_old = f_new

In [ ]:
v_cf

In [51]:
#VACF Plot
plt.plot(v_cf)
plt.xlabel('Timesteps(t0)')
plt.ylabel('VACF')

In [ ]:
#fourier transform of VACF for DOS
def autocorr(X):
    """ the convolution is actually being done here
    meaning from -inf to inf so we only want half the
    array"""

    result = np.correlate(X, X, mode='full')
    return result[result.size/2:]

    
def fft_autocorr(AutoCorr,dt):
    """FFT of autocorrelation function"""
    #fft_arry = fftpack.dct(AutoCorr)*dt
    fft_arry = np.fft.rfft(AutoCorr)*dt
    return fft_arry


In [ ]:
#Plot Discrete FT of VACF
#runtime total
dt = 0.005E-12*100
#size of frames
vsize=100
fft_v = fft_autocorr(v_cf,dt) * 1.0E12 #THz^-1     
freq = np.fft.rfftfreq(vsize, d=dt) /(2* 1.0E12) #THz
plt.plot(freq,np.abs(fft_v))
plt.xlabel('Freq. (THz)')
plt.ylabel('DOS')